<a href="https://colab.research.google.com/github/anubhavshrestha/sift_feature_extraction/blob/main/CIFAR_10_classical_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim


transforms_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transforms_train)

print(trainset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers = 0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transforms_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers = 0)

class Net(nn.Module):
    def __init__ (self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,64,4, stride=1, padding=2)
        self.batch_normalize = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64,64,4, stride=1, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(p=0.5)
        self.conv3 = nn.Conv2d(64,64,4, stride=1, padding=2)
        self.conv4 = nn.Conv2d(64,64,4, stride=1, padding=2)
        self.conv5 = nn.Conv2d(64,64,4, stride=1, padding=2)
        self.conv6 = nn.Conv2d(64,64,3, stride=1, padding=0)
        self.conv7 = nn.Conv2d(64,64,3, stride=1, padding=0)
        self.conv8 = nn.Conv2d(64,64,3, stride=1, padding=0)
        self.full_conn_1 = nn.Linear(64*4*4, 500)
        self.full_conn_2 = nn.Linear(500, 500)
        self.full_conn_3 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.batch_normalize(F.relu(self.conv1(x)))
        x = self.drop(self.pool(F.relu(self.conv2(x))))
        x = self.batch_normalize(F.relu(self.conv3(x)))
        x = self.drop(self.pool(F.relu(self.conv4(x))))
        x = self.batch_normalize(F.relu(self.conv5(x)))
        x = self.drop(F.relu(self.conv6(x)))
        x = self.batch_normalize(F.relu(self.conv7(x)))
        x = self.drop(self.batch_normalize(F.relu(self.conv8(x))))
        x = x.view(-1, 64*4*4)
        x = F.relu(self.full_conn_1(x))
        x = F.relu(self.full_conn_2(x))
        x = self.full_conn_3(x)
        return x

net = Net()
net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# training the model

for epoch in range(10):  # loop over the dataset multiple times

    total_right = 0
    total = 0

    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        total_right += (predicted == labels.data).float().sum()

    print("Training Accuracy for epoch {} : {}".format(epoch+1,total_right/total))

    if (epoch+1) % 5 == 0:
        torch.save(net, 'save_params.ckpt')

# test the model

my_model = torch.load('save_params.ckpt')

total_right = 0
total = 0


with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = Variable(images).cuda(), Variable(labels).cuda()
        outputs = my_model(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        total_right += (predicted == labels.data).float().sum()

print('Test accuracy: %d %%' % (
    100 * total_right / total))





100%|██████████| 170498071/170498071 [00:03<00:00, 48813905.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Files already downloaded and verified
Training Accuracy for epoch 1 : 0.3766799867153168
Training Accuracy for epoch 2 : 0.5324199795722961
Training Accuracy for epoch 3 : 0.606440007686615
Training Accuracy for epoch 4 : 0.6466599702835083
Training Accuracy for epoch 5 : 0.6696599721908569
Training Accuracy for epoch 6 : 0.6917799711227417
Training Accuracy for epoch 7 : 0.7078999876976013
Training Accuracy for epoch 8 : 0.7247399687767029
Training Accuracy for epoch 9 : 0.7340999841690063
Training Accuracy for epoch 10 : 0.7446199655532837
Test accuracy: 74 %
